# Linear Modeling

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
housing_train = pd.read_csv('housing_train.csv')
housing_test = pd.read_csv('housing_test.csv')

y_housing = pd.read_csv('y_housing.csv')

In [ ]:
print('Full Train dimension:', housing_train.shape)
print('Y train dimension:', y_housing.shape)
print('Test data dimension:', housing_test.shape)



In [ ]:
from sklearn import linear_model
from sklearn.linear_model import ElasticNet, Lasso, Ridge, SGDRegressor
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import  train_test_split
from sklearn.metrics import mean_squared_error, log_loss

import sklearn.model_selection as ms

from sklearn import ensemble
from sklearn.ensemble import RandomForestRegressor
from pprint import pprint
from sklearn.svm import SVR

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(housing_train, y_housing, test_size = 0.30, random_state = 42)

In [ ]:
ridge = linear_model.Ridge(alpha = 1, normalize=True)
lasso = linear_model.Lasso(alpha= 0, normalize = True)
elasticnet = linear_model.ElasticNet(alpha = 0.01, l1_ratio=0.5, normalize=False)
clf = linear_model.SGDRegressor(max_iter=1000, tol=1e-3)

gbm = ensemble.GradientBoostingRegressor()
rf = ensemble.RandomForestRegressor(n_estimators = 800, min_samples_split=5,
                                    min_samples_leaf = 1, max_features = 'sqrt',
                                    max_depth = 92, bootstrap = False, random_state=42)



In [ ]:
housing_train.shape

In [ ]:
y_housing.shape

In [ ]:
modelList =[ridge, lasso, elasticnet, clf, gbm, rf]
modelSeries= pd.Series(modelList, index =[ 'Ridge', 'Lasso', 'Elasticnet', 'SGD', 'XGboost', 'Random Forest'])



In [ ]:
modelSeries.apply(lambda x:x.fit(X_train, y_train))

In [ ]:
ans = pd.concat([modelSeries.apply(lambda x: x.score(X_train,y_train)),modelSeries.apply(lambda x: x.score(X_test,y_test)), 
                                    modelSeries.apply(lambda x: np.sqrt(mean_squared_error(x.predict(X_test), y_test)))],axis=1)

ans.columns = ['train score', 'test score', 'rmse']
ans

In [ ]:
###################################################################################################################

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn import utils

lab_enc = preprocessing.LabelEncoder()
y_test_encoded = lab_enc.fit_transform(y_test)
print(y_test_encoded)

print(utils.multiclass.type_of_target(y_test))
print(utils.multiclass.type_of_target(y_test.astype('int')))
print(utils.multiclass.type_of_target(y_test_encoded))

In [ ]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix
mistakes = 0 # the mistakes each individual tree makes
confusion_pair = 0

n_pairs = 0.5 * rf.n_estimators * (rf.n_estimators-1)
for idx, tree in enumerate(rf.estimators_):
    mistakes += confusion_matrix(tree.predict(X_test), y_test)
    for idx2, tree2 in enumerate(rf.estimators_):
            if idx2 == idx: continue
            confusion_pair += confusion_matrix(tree.predict(X_test), tree2.predict(X_test))   
print("Average per Tree Confusion:")
print(mistakes/rf.n_estimators*1.0)
print("Aggregate Pairwise Confusion:")
print(confusion_pair/n_pairs)

In [ ]:
en_prediction = rf.predict(X_test)
en_mse = mean_squared_error(en_prediction, y_test)
en_rsme = np.sqrt(en_mse)
print('RF RMSE: %.2f' % en_rsme)

## Hyperparameter tuning RF

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
print('Parameters currently in use:\n')
pprint(rf.get_params())

In [ ]:
###### number of trees in random forest
n_estimators =[int(x) for x in np.linspace(start =200, stop = 
                                           2000, num=10)]

In [ ]:
##### Number of features to consider at every split
max_features = ['auto', 'sqrt']

In [ ]:
#### Max number of levels in tree
max_depth = [int(x) for x in np.linspace(20, 110, num=11)]
max_depth.append(None)

In [ ]:
###### min samples split a node
min_samples_split = [2, 5, 10]

In [ ]:
#### Minimum number of samples required to split a node
min_samples_leaf = [1, 2, 4]

In [ ]:
#### Method of selecting samples for training each tree
bootstrap = [True, False]

In [ ]:
#### Create random grid
random_grid ={'n_estimators':n_estimators,
             'max_features': max_features,
             'max_depth': max_depth,
             'min_samples_split': min_samples_split,
             'min_samples_leaf': min_samples_leaf,
             'bootstrap':bootstrap}
pprint(random_grid)

In [ ]:
#### Using random grid to search for best hyperparameters
### base mode
rf_random = RandomizedSearchCV(estimator = rf, param_distributions =
                              random_grid, n_iter = 100, cv = 3, verbose =2,
                              random_state=42,
                              n_jobs=-1)

In [ ]:
#### Value error fix for refit model into rf_random

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn import utils

lab_enc = preprocessing.LabelEncoder()
y_train_encoded = lab_enc.fit_transform(y_train)
print(y_train_encoded)

print(utils.multiclass.type_of_target(y_train))
print(utils.multiclass.type_of_target(y_train.astype('int')))
print(utils.multiclass.type_of_target(y_train_encoded))

### New Model Fitting

In [ ]:
rf_random.fit(X_train, y_train_encoded)

In [ ]:
#### best parameters
rf_random.best_params_

### Ridge hyper parameters

In [ ]:
from scipy.stats import uniform as sp_rand
print('Parameters currently in use:\n')
pprint(ridge.get_params())

In [ ]:
param_grid ={'alpha':sp_rand()}
model = Ridge()
research = RandomizedSearchCV(estimator=model, param_distributions=param_grid,
                             n_iter = 100)

In [ ]:
research.fit(X_train, y_train)
print(research)

In [ ]:
print(research.best_score_)
print(research.best_estimator_.alpha)